<a href="https://colab.research.google.com/github/FurqanBhat/ML-Colab-Notebooks/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [76]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [77]:
input_size=28
sequence_length=28
hidden_size=128
num_classes=10
num_epochs=2
batch_size=100
learnig_rate=0.001
num_layers=2

In [78]:
train_dataset= torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset=torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

train_loader=torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader=torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [79]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.num_layers=num_layers
    self.hidden_size=hidden_size
    self.rnn=nn.RNN(input_size=input_size, hidden_size=hidden_size,
                    num_layers=num_layers, batch_first=True)
    self.fc=nn.Linear(hidden_size, num_classes)


  def forward(self, x):
    h0=torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    out, _ =self.rnn(x,h0)
    #batch_sizeXsequence_lengthXhidden_size
    #out: N, 28, 128

    out=out[:,-1, :]
    out=self.fc(out)
    return out



In [80]:
model=RNN(input_size,hidden_size, num_layers, num_classes).to(device)

In [81]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(), lr=learnig_rate)

In [82]:
n_total_steps=len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images=images.reshape(-1, sequence_length, input_size).to(device)
    labels=labels.to(device)

    outputs=model(images)
    loss=criterion(outputs, labels)


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    if (i+1)%100==0:
      print(f"Epoch {epoch+1}/{num_epochs} , Step {i+1}/{n_total_steps}, Loss:   {loss.item():.4f}")



Epoch 1/2 , Step 100/600, Loss:   0.9919
Epoch 1/2 , Step 200/600, Loss:   0.6174
Epoch 1/2 , Step 300/600, Loss:   0.6183
Epoch 1/2 , Step 400/600, Loss:   0.4330
Epoch 1/2 , Step 500/600, Loss:   0.2557
Epoch 1/2 , Step 600/600, Loss:   0.2875
Epoch 2/2 , Step 100/600, Loss:   0.3831
Epoch 2/2 , Step 200/600, Loss:   0.2808
Epoch 2/2 , Step 300/600, Loss:   0.3560
Epoch 2/2 , Step 400/600, Loss:   0.1661
Epoch 2/2 , Step 500/600, Loss:   0.2357
Epoch 2/2 , Step 600/600, Loss:   0.1906


In [83]:
with torch.no_grad():
  n_samples=len(test_loader.dataset)
  n_correct=0
  for images, labels in test_loader:
    images=images.reshape(-1,sequence_length, input_size).to(device)
    labels=labels.to(device)
    outputs=model(images)

    _, predicted=torch.max(outputs, 1)
    n_correct+=(predicted==labels).sum().item()


  print(f"accuracy={n_correct/n_samples}")






accuracy=0.942
